# DeepCTRL Healthcare
Este es un notebook que descompone en celdas el código de la red neuronal basada en reglas DeepCTRL para poder comprender mejor la integración de reglas dentro de una red neuronal.
La regla asociada con esta aplicación es de **lógica asociativa**.
En el código original (**train.py**) se importan funciones de dos módulos personalizados que contienen distintas funciones. Estos módulos son **utils_learning.py** y **utils_cardio.py**.

# Librerías necesarias

In [1]:
import os

from copy import deepcopy
import pandas as pd
import numpy as np
import random

from sklearn.preprocessing import OneHotEncoder, Normalizer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score, precision_score, recall_score, precision_recall_curve, accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.distributions.beta import Beta

# Funciones de utils_learning

In [2]:
def verification(out, pert_out, threshold=0.0):
    '''
    return the ratio of qualified samples.
    '''
    if isinstance(out, torch.Tensor):
        return 1.0*torch.sum(pert_out-out < threshold) / out.shape[0]
    else:
        return 1.0*np.sum(pert_out-out < threshold) / out.shape[0]

In [3]:
def get_perturbed_input(input_tensor, pert_coeff):
    '''
    X = X + pert_coeff*rand*X
    return input_tensor + input_tensor*pert_coeff*torch.rand()
    '''
    device = input_tensor.device
    return input_tensor + torch.abs(input_tensor)*pert_coeff*torch.rand(input_tensor.shape, device=device)

# Parámetros necesarios para la ejecución de la red
parser.add_argument('--datapath', type=str, default='data/cardio_train.csv')<br>
parser.add_argument('--rule_threshold', type=float, default=129.5)<br>
parser.add_argument('--src_usual_ratio', type=float, default=0.3)<br>
parser.add_argument('--src_unusual_ratio', type=float, default=0.7)<br>
parser.add_argument('--seed', type=int, default=42)<br>
parser.add_argument('--device', type=str, default='cuda:0')<br>
parser.add_argument('--target_rule_ratio', type=float, default=0.7)<br>
parser.add_argument('--batch_size', type=int, default=32)<br>
parser.add_argument('--train_ratio', type=float, default=0.7)<br>
parser.add_argument('--validation_ratio', type=float, default=0.1)<br>
parser.add_argument('--test_ratio', type=float, default=0.2)<br>
parser.add_argument('--model_type', type=str, default='dataonly')<br>
parser.add_argument('--input_dim_encoder', type=int, default=16)<br>
parser.add_argument('--output_dim_encoder', type=int, default=16)<br>
parser.add_argument('--hidden_dim_encoder', type=int, default=100)<br>
parser.add_argument('--hidden_dim_db', type=int, default=16)<br>
parser.add_argument('--n_layers', type=int, default=1)<br>
parser.add_argument('--rule_ind', type=int, default=5)  <br>
parser.add_argument('--epochs', type=int, default=1000, help='default: 1000')<br>
parser.add_argument('--early_stopping_thld', type=int, default=10, help='default: 10')<br>
parser.add_argument('--valid_freq', type=int, default=1, help='default: 1')<br>

In [4]:
# Declaración manual de los parámetros para la ejecución de la red
datapath = 'controllabledl/healthcare/data/cardio_train.csv'
rule_threshold = 129.5
src_usual_ratio = 0.3 # Porcentaje de los pacientes etiquetados como usuales usar para el entrenamiento
src_unusual_ratio = 0.7 # Porcentaje de los pacientes etiquetados como inusuales a usar para el entrenamiento
seed = 42
device = 'cuda'
target_rule_ratio = 0.7
batch_size = 32
train_ratio = 0.7
validation_ratio = 0.1
test_ratio = 0.2
model_type = 'ours-beta0.1-scale0.01' # Esta línea declara qué tipo de red se utilizará, si una que aprenda solo de los datos o en conjunto con las reglas. En la siguiente celda anoto los posibles valores para esta variable.
input_dim_encoder = 16
output_dim_encoder = 16
hidden_dim_encoder = 100
hidden_dim_db = 16
n_layers = 1
rule_ind = 5
epochs = 100
early_stopping_thld = 20
valid_freq = 1


In [5]:
# Estos son los distintos tipos de configuraciones que podemos probar añadiendo el argumento --model_type o en el caso de este JupyterNotebook, modificar la variable
# model_type de la celda de arriba
model_info = {'dataonly': {'rule': 0.0}, 
              'ours-beta1.0': {'beta': [1.0], 'scale': 1.0, 'lr': 0.001},
              'ours-beta0.1': {'beta': [0.1], 'scale': 1.0, 'lr': 0.001},
              'ours-beta0.1-scale0.1': {'beta': [0.1], 'scale': 0.1},
              'ours-beta0.1-scale0.01': {'beta': [0.1], 'scale': 0.01},
              'ours-beta0.1-scale0.05': {'beta': [0.1], 'scale': 0.05},
              'ours-beta0.1-pert0.001': {'beta': [0.1], 'pert': 0.001},
              'ours-beta0.1-pert0.01': {'beta': [0.1], 'pert': 0.01},
              'ours-beta0.1-pert0.1': {'beta': [0.1], 'pert': 0.1},
              'ours-beta0.1-pert1.0': {'beta': [0.1], 'pert': 1.0},
             }  

# Construcción de los bloques que componen a la red neuronal

In [6]:
class NaiveModel(nn.Module):
  def __init__(self):
    super(NaiveModel, self).__init__()
    self.net = nn.Identity()

  def forward(self, x, alpha=0.0):
    return self.net(x)

In [7]:
class RuleEncoder(nn.Module):
  def __init__(self, input_dim, output_dim, hidden_dim=4):
    super(RuleEncoder, self).__init__()
    self.input_dim = input_dim
    self.output_dim = output_dim
    self.net = nn.Sequential(nn.Linear(input_dim, hidden_dim),
                             nn.ReLU(),
                             nn.Linear(hidden_dim, output_dim))

  def forward(self, x):
    return self.net(x)

In [8]:
class DataEncoder(nn.Module):
  def __init__(self, input_dim, output_dim, hidden_dim=4):
    super(DataEncoder, self).__init__()
    self.input_dim = input_dim
    self.output_dim = output_dim
    self.net = nn.Sequential(nn.Linear(input_dim, hidden_dim),
                             nn.ReLU(),
                             nn.Linear(hidden_dim, output_dim))

  def forward(self, x):
    return self.net(x)

## Red que integra el codificador de relgas y de datos

In [9]:
class Net(nn.Module):
  def __init__(self, input_dim, output_dim, rule_encoder, data_encoder, hidden_dim=4, n_layers=2, merge='cat', skip=False, input_type='state'):
    super(Net, self).__init__()
    self.skip = skip
    self.input_type = input_type
    self.rule_encoder = rule_encoder
    self.data_encoder = data_encoder
    self.n_layers = n_layers
    assert self.rule_encoder.input_dim == self.data_encoder.input_dim
    assert self.rule_encoder.output_dim == self.data_encoder.output_dim
    self.merge = merge
    if merge == 'cat':
      self.input_dim_decision_block = self.rule_encoder.output_dim * 2
    elif merge == 'add':
      self.input_dim_decision_block = self.rule_encoder.output_dim

    self.net = []
    for i in range(n_layers):
      if i == 0:
        in_dim = self.input_dim_decision_block
      else:
        in_dim = hidden_dim

      if i == n_layers-1:
        out_dim = output_dim
      else:
        out_dim = hidden_dim

      self.net.append(nn.Linear(in_dim, out_dim))
      if i != n_layers-1:
        self.net.append(nn.ReLU())

    self.net.append(nn.Sigmoid())

    self.net = nn.Sequential(*self.net)

  def get_z(self, x, alpha=0.0):
    rule_z = self.rule_encoder(x)
    data_z = self.data_encoder(x)

    if self.merge == 'add':
      z = alpha*rule_z + (1-alpha)*data_z
    elif self.merge == 'cat':
      z = torch.cat((alpha*rule_z, (1-alpha)*data_z), dim=-1)
    elif self.merge == 'equal_cat':
      z = torch.cat((rule_z, data_z), dim=-1)

    return z

  def forward(self, x, alpha=0.0):
    # merge: cat or add

    rule_z = self.rule_encoder(x)
    data_z = self.data_encoder(x)

    if self.merge == 'add':
      z = alpha*rule_z + (1-alpha)*data_z
    elif self.merge == 'cat':
      z = torch.cat((alpha*rule_z, (1-alpha)*data_z), dim=-1) # Aquí ocurre la unión estocástica de las representaciones latentes de los encoders de reglas y de datos
    elif self.merge == 'equal_cat':
      z = torch.cat((rule_z, data_z), dim=-1)

    if self.skip:
      if self.input_type == 'seq':
        return self.net(z) + x[:, -1, :]
      else:
        return self.net(z) + x    # predict delta values
    else:
      return self.net(z)    # predict absolute values 

# Proceso de entrenamiento

In [10]:
# Definición de parámetros para reproducibilidad
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [11]:
# Load dataset
df = pd.read_csv(os.path.join(datapath), delimiter=';')
df = df.drop(['id'], axis=1)

y = df['cardio'] # y representa la etiqueta, o ground truth
X_raw = df.drop(['cardio'], axis=1) # X_raw representa solo los datos sin etiquetar

print("Target class ratio:")
print("# of cardio=1: {}/{} ({:.2f}%)".format(np.sum(y==1), len(y), 100*np.sum(y==1)/len(y)))
print("# of cardio=0: {}/{} ({:.2f}%)\n".format(np.sum(y==0), len(y), 100*np.sum(y==0)/len(y)))

Target class ratio:
# of cardio=1: 34979/70000 (49.97%)
# of cardio=0: 35021/70000 (50.03%)



In [12]:
column_trans = ColumnTransformer(
[('age_norm', StandardScaler(), ['age']),
('height_norm', StandardScaler(), ['height']),
('weight_norm', StandardScaler(), ['weight']),
('gender_cat', OneHotEncoder(), ['gender']),
('ap_hi_norm', StandardScaler(), ['ap_hi']),
('ap_lo_norm', StandardScaler(), ['ap_lo']),
('cholesterol_cat', OneHotEncoder(), ['cholesterol']),
('gluc_cat', OneHotEncoder(), ['gluc']),
('smoke_cat', OneHotEncoder(), ['smoke']),
('alco_cat', OneHotEncoder(), ['alco']),
('active_cat', OneHotEncoder(), ['active']),
], remainder='passthrough'
)

X = column_trans.fit_transform(X_raw)
num_samples = X.shape[0]
X_np = X.copy()

## Regla

In [13]:
# ---------- RULE
# Rule : higher ap -> higher risk. Aquí se trata de un regla de lógica.
rule_threshold = rule_threshold
rule_ind = rule_ind
rule_feature = 'ap_hi'

low_ap_negative = (df[rule_feature] <= rule_threshold) & (df['cardio'] == 0)    # usual
high_ap_positive = (df[rule_feature] > rule_threshold) & (df['cardio'] == 1)    # usual
low_ap_positive = (df[rule_feature] <= rule_threshold) & (df['cardio'] == 1)    # unusual
high_ap_negative = (df[rule_feature] > rule_threshold) & (df['cardio'] == 0)    # unusual
# ---------- END OF RULE 

## Construcción del dataset

In [14]:
# Samples in Usual group
X_usual = X[low_ap_negative | high_ap_positive]
y_usual = y[low_ap_negative | high_ap_positive]
y_usual = y_usual.to_numpy()
X_usual, y_usual = shuffle(X_usual, y_usual, random_state=0)
num_usual_samples = X_usual.shape[0]

# Samples in Unusual group
X_unusual = X[low_ap_positive | high_ap_negative]
y_unusual = y[low_ap_positive | high_ap_negative]
y_unusual = y_unusual.to_numpy()
X_unusual, y_unusual = shuffle(X_unusual, y_unusual, random_state=0)
num_unusual_samples = X_unusual.shape[0]

# Build a source dataset
src_usual_ratio = src_usual_ratio
src_unusual_ratio = src_unusual_ratio
num_samples_from_unusual = int(src_unusual_ratio * num_unusual_samples)
num_samples_from_usual = int(num_samples_from_unusual * src_usual_ratio / (1-src_usual_ratio))

X_src = np.concatenate((X_usual[:num_samples_from_usual], X_unusual[:num_samples_from_unusual]), axis=0)
y_src = np.concatenate((y_usual[:num_samples_from_usual], y_unusual[:num_samples_from_unusual]), axis=0)
print()
print("Source dataset statistics:")
print("# of samples in Usual group: {}".format(num_samples_from_usual))
print("# of samples in Unusual group: {}".format(num_samples_from_unusual))
print("Usual ratio: {:.2f}%".format(100 * num_samples_from_usual / (X_src.shape[0])))


Source dataset statistics:
# of samples in Usual group: 6007
# of samples in Unusual group: 14018
Usual ratio: 30.00%


## Construcción de los sets de entrenamiento y validación

In [15]:
train_ratio = train_ratio
validation_ratio = validation_ratio
test_ratio = test_ratio
train_X, test_X, train_y, test_y = train_test_split(X_src, y_src, test_size=1 - train_ratio, random_state=seed)
valid_X, test_X, valid_y, test_y = train_test_split(test_X, test_y, test_size=test_ratio / (test_ratio + validation_ratio), random_state=seed)

train_X, train_y = torch.tensor(train_X, dtype=torch.float32, device=device), torch.tensor(train_y, dtype=torch.float32, device=device)
valid_X, valid_y = torch.tensor(valid_X, dtype=torch.float32, device=device), torch.tensor(valid_y, dtype=torch.float32, device=device)
test_X, test_y = torch.tensor(test_X, dtype=torch.float32, device=device), torch.tensor(test_y, dtype=torch.float32, device=device)

batch_size = batch_size
train_loader = DataLoader(TensorDataset(train_X, train_y), batch_size=batch_size, shuffle=True) # Aquí se crea el trainset a partir de revolver los datos en las líneas 172 a 174
valid_loader = DataLoader(TensorDataset(valid_X, valid_y), batch_size=valid_X.shape[0])
test_loader = DataLoader(TensorDataset(test_X, test_y), batch_size=test_X.shape[0])
print("data size: {} for training / {} for validation / {} for testing".format(len(train_X), len(valid_X), len(test_X)))

data size: 14017 for training / 2002 for validation / 4006 for testing


## Especificaciones del modelo a entrenar

In [16]:
model_type = model_type
if model_type not in model_info:
    # default setting
    lr = 0.001
    pert_coeff = 0.1
    scale = 1.0
    beta_param = [1.0]
    alpha_distribution = Beta(float(beta_param[0]), float(beta_param[0]))
    model_params = {}

else:
    model_params = model_info[model_type] # Aquí se toma información para los parámetros de alpha y de influencia de las reglas
    lr = model_params['lr'] if 'lr' in model_params else 0.001
    pert_coeff = model_params['pert'] if 'pert' in model_params else 0.1 # Definición del coeficiente de perturbación
    scale = model_params['scale'] if 'scale' in model_params else 1.0
    beta_param = model_params['beta'] if 'beta' in model_params else [1.0]

if len(beta_param) == 1:
    alpha_distribution = Beta(float(beta_param[0]), float(beta_param[0]))
elif len(beta_param) == 2:
    alpha_distribution = Beta(float(beta_param[0]), float(beta_param[1]))

print('model_type: {}\tscale:{}\tBeta distribution: Beta({})\tlr: {}\t \tpert_coeff: {}'.format(model_type, scale, beta_param, lr, pert_coeff))

model_type: ours-beta0.1-scale0.01	scale:0.01	Beta distribution: Beta([0.1])	lr: 0.001	 	pert_coeff: 0.1


## Declaración de hiperparámetros de la red

In [17]:
# Aquellas variables que sean iguales a sí mismas (epochs = epochs) es porque se les quitó 'args.' para la modificación de parámetros desde terminal
merge = 'cat'
input_dim = 19
output_dim_encoder = output_dim_encoder # Default 16
hidden_dim_encoder = hidden_dim_encoder # Default 16
hidden_dim_db = hidden_dim_db
n_layers = n_layers
output_dim = 1

rule_encoder = RuleEncoder(input_dim, output_dim_encoder, hidden_dim_encoder)
data_encoder = DataEncoder(input_dim, output_dim_encoder, hidden_dim_encoder)
model = Net(input_dim, output_dim, rule_encoder, data_encoder, hidden_dim=hidden_dim_db, n_layers=n_layers, merge=merge).to(device)    # Not residual connection

optimizer = optim.Adam(model.parameters(), lr=lr)        
loss_rule_func = lambda x,y: torch.mean(F.relu(x-y))    # if x>y, penalize it.
loss_task_func = nn.BCELoss()    # return scalar (reduction=mean)

epochs = epochs
early_stopping_thld = early_stopping_thld
counter_early_stopping = 1
valid_freq = valid_freq  

saved_filename = 'cardio_{}_rule-{}_src{}-target{}_seed{}.demo.pt'.format(model_type, rule_feature, src_usual_ratio, src_usual_ratio, seed)
saved_filename =  os.path.join('controllabledl/healthcare/saved_models', saved_filename)
print('saved_filename: {}\n'.format(saved_filename))
best_val_loss = float('inf')

saved_filename: controllabledl/healthcare/saved_models/cardio_ours-beta0.1-scale0.01_rule-ap_hi_src0.3-target0.3_seed42.demo.pt



## Proceso de entrenamiento

In [18]:
for epoch in range(1, epochs+1):
    model.train()
    for batch_train_x, batch_train_y in train_loader:
        batch_train_y = batch_train_y.unsqueeze(-1)

        optimizer.zero_grad()

        if model_type.startswith('dataonly'):
            alpha = 0.0
        elif model_type.startswith('ruleonly'):
            alpha = 1.0
        elif model_type.startswith('ours'):
            alpha = alpha_distribution.sample().item()

        # stable output
        output = model(batch_train_x, alpha=alpha)
        loss_task = loss_task_func(output, batch_train_y)

        # perturbed input and its output
        pert_batch_train_x = batch_train_x.detach().clone()
        pert_batch_train_x[:,rule_ind] = get_perturbed_input(pert_batch_train_x[:,rule_ind], pert_coeff) # Necesita de la función get_perturbed_input en el módulo
        # utils_learning.py
        pert_output = model(pert_batch_train_x, alpha=alpha)

        loss_rule = loss_rule_func(output, pert_output)    # output should be less than pert_output

        loss = alpha * loss_rule + scale * (1 - alpha) * loss_task

        loss.backward()
        optimizer.step()

    # Evaluate on validation set
    if epoch % valid_freq == 0:
        model.eval()
        if  model_type.startswith('ruleonly'):
            alpha = 1.0
        else:
            alpha = 0.0

        with torch.no_grad():
            for val_x, val_y in valid_loader:
                val_y = val_y.unsqueeze(-1)

                output = model(val_x, alpha=alpha)
                val_loss_task = loss_task_func(output, val_y).item()

                # perturbed input and its output
                pert_val_x = val_x.detach().clone()
                pert_val_x[:,rule_ind] = get_perturbed_input(pert_val_x[:,rule_ind], pert_coeff)
                pert_output = model(pert_val_x, alpha=alpha)    # \hat{y}_{p}    predicted sales from perturbed input

                val_loss_rule = loss_rule_func(output, pert_output).item()
                val_ratio = verification(pert_output, output, threshold=0.0).item()

                val_loss = val_loss_task

                y_true = val_y.cpu().numpy()
                y_score = output.cpu().numpy()
                y_pred = np.round(y_score)
                val_acc = 100 * accuracy_score(y_true, y_pred)

        if val_loss < best_val_loss:
            counter_early_stopping = 1
            best_val_loss = val_loss
            best_model_state_dict = deepcopy(model.state_dict())
            print('[Valid] Epoch: {} Loss: {:.6f} (alpha: {:.2f})\t Loss(Task): {:.6f} Acc: {:.2f}\t Loss(Rule): {:.6f}\t Ratio: {:.4f} best model is updated %%%%'
                .format(epoch, best_val_loss, alpha, val_loss_task, val_acc, val_loss_rule, val_ratio))
            torch.save({
                'epoch': epoch,
                'model_state_dict': best_model_state_dict,
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': best_val_loss
            }, saved_filename)
        else:
            print('[Valid] Epoch: {} Loss: {:.6f} (alpha: {:.2f})\t Loss(Task): {:.6f} Acc: {:.2f}\t Loss(Rule): {:.6f}\t Ratio: {:.4f}({}/{})'
                .format(epoch, val_loss, alpha, val_loss_task, val_acc, val_loss_rule, val_ratio, counter_early_stopping, early_stopping_thld))
            if counter_early_stopping >= early_stopping_thld:
                break
            else:
                counter_early_stopping += 1

[Valid] Epoch: 1 Loss: 0.658670 (alpha: 0.00)	 Loss(Task): 0.658670 Acc: 61.19	 Loss(Rule): 0.000306	 Ratio: 0.0000 best model is updated %%%%
[Valid] Epoch: 2 Loss: 0.651448 (alpha: 0.00)	 Loss(Task): 0.651448 Acc: 61.24	 Loss(Rule): 0.000522	 Ratio: 0.0000 best model is updated %%%%
[Valid] Epoch: 3 Loss: 0.649247 (alpha: 0.00)	 Loss(Task): 0.649247 Acc: 61.24	 Loss(Rule): 0.000696	 Ratio: 0.0005 best model is updated %%%%
[Valid] Epoch: 4 Loss: 0.647123 (alpha: 0.00)	 Loss(Task): 0.647123 Acc: 60.84	 Loss(Rule): 0.000884	 Ratio: 0.0000 best model is updated %%%%
[Valid] Epoch: 5 Loss: 0.647068 (alpha: 0.00)	 Loss(Task): 0.647068 Acc: 60.44	 Loss(Rule): 0.001154	 Ratio: 0.0000 best model is updated %%%%
[Valid] Epoch: 6 Loss: 0.645463 (alpha: 0.00)	 Loss(Task): 0.645463 Acc: 61.64	 Loss(Rule): 0.001310	 Ratio: 0.0000 best model is updated %%%%
[Valid] Epoch: 7 Loss: 0.639192 (alpha: 0.00)	 Loss(Task): 0.639192 Acc: 61.89	 Loss(Rule): 0.001598	 Ratio: 0.0000 best model is updated %%%%

## Test the network

In [19]:
# Test
rule_encoder = RuleEncoder(input_dim, output_dim_encoder, hidden_dim_encoder)
data_encoder = DataEncoder(input_dim, output_dim_encoder, hidden_dim_encoder)
model_eval = Net(input_dim, output_dim, rule_encoder, data_encoder, hidden_dim=hidden_dim_db, n_layers=n_layers, merge=merge).to(device)    # Not residual connection

# Aquí es donde se carga el modelo con mejor rendimiento obtenido durante el entrenamiento
checkpoint = torch.load(saved_filename) 
model_eval.load_state_dict(checkpoint['model_state_dict'])
print("best model loss: {:.6f}\t at epoch: {}".format(checkpoint['loss'], checkpoint['epoch']))

model_eval.eval()
with torch.no_grad():
   for te_x, te_y in test_loader:
       te_y = te_y.unsqueeze(-1)

output = model_eval(te_x, alpha=0.0)
test_loss_task = loss_task_func(output, te_y).item()
print('\n[Test] Average loss: {:.8f}\n'.format(test_loss_task))

model_eval.eval()
alphas = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
# perturbed input and its output
pert_test_x = te_x.detach().clone()
pert_test_x[:,rule_ind] = get_perturbed_input(pert_test_x[:,rule_ind], pert_coeff)
for alpha in alphas:
    model_eval.eval()
    with torch.no_grad():
      for te_x, te_y in test_loader:
        te_y = te_y.unsqueeze(-1)

      if model_type.startswith('dataonly'):
        output = model_eval(te_x, alpha=0.0)
      elif model_type.startswith('ours'):
        output = model_eval(te_x, alpha=alpha)
      elif model_type.startswith('ruleonly'):
        output = model_eval(te_x, alpha=1.0)

      test_loss_task = loss_task_func(output, te_y).item()

      if model_type.startswith('dataonly'):
        pert_output = model_eval(pert_test_x, alpha=0.0)
      elif model_type.startswith('ours'):
        pert_output = model_eval(pert_test_x, alpha=alpha)
      elif model_type.startswith('ruleonly'):
        pert_output = model_eval(pert_test_x, alpha=1.0)

      test_ratio = verification(pert_output, output, threshold=0.0).item()

      y_true = te_y.cpu().numpy()
      y_score = output.cpu().numpy()
      y_pred = np.round(y_score)
      test_acc = accuracy_score(y_true, y_pred)

    print('[Test] Average loss: {:.8f} (alpha:{})'.format(test_loss_task, alpha))
    print('[Test] Accuracy: {:.4f} (alpha:{})'.format(test_acc, alpha))
    print("[Test] Ratio of verified predictions: {:.6f} (alpha:{})".format(test_ratio, alpha))
    print()

best model loss: 0.585012	 at epoch: 61

[Test] Average loss: 0.59583944

[Test] Average loss: 0.59583944 (alpha:0.0)
[Test] Accuracy: 0.7002 (alpha:0.0)
[Test] Ratio of verified predictions: 0.498502 (alpha:0.0)

[Test] Average loss: 0.59789699 (alpha:0.1)
[Test] Accuracy: 0.6912 (alpha:0.1)
[Test] Ratio of verified predictions: 0.575886 (alpha:0.1)

[Test] Average loss: 0.60234100 (alpha:0.2)
[Test] Accuracy: 0.6882 (alpha:0.2)
[Test] Ratio of verified predictions: 0.627309 (alpha:0.2)

[Test] Average loss: 0.60927165 (alpha:0.3)
[Test] Accuracy: 0.6767 (alpha:0.3)
[Test] Ratio of verified predictions: 0.684723 (alpha:0.3)

[Test] Average loss: 0.61878848 (alpha:0.4)
[Test] Accuracy: 0.6658 (alpha:0.4)
[Test] Ratio of verified predictions: 0.739391 (alpha:0.4)

[Test] Average loss: 0.63099283 (alpha:0.5)
[Test] Accuracy: 0.6528 (alpha:0.5)
[Test] Ratio of verified predictions: 0.779581 (alpha:0.5)

[Test] Average loss: 0.64598787 (alpha:0.6)
[Test] Accuracy: 0.6313 (alpha:0.6)
[Test]

## Para graficar rendimiento (no completado)

In [ ]:
def get_metrics(y_true, y_pred, y_score):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)
    
    return acc, prec, recall, fpr, tpr, roc_auc

def get_correct_results(out, label_Y):
    y_pred_tag = torch.round(out)    # Binary label
    return (y_pred_tag == label_Y).sum().float()

In [ ]:
acc, prec, recall,fpr, tpr, roc_auc = get_metrics(y_true,y_pred,y_score)